In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import itertools
import sys
from pathlib import Path

import numpy as np

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from vibromaf import config
from vibromaf.util import matlab, model
from vibromaf.util.common import print_metric, print_mse_and_pc

In [ ]:
test_indices = [
    0,  # Aluminum Grid - fast
    5,  # Polyester Pad - slower
]

nsnr_train, nsnr_test = matlab.load_data_for_metric("NSNR", test_indices)
sim_train, sim_test = matlab.load_data_for_metric("SIM", test_indices)
spqi_train, spqi_test = matlab.load_data_for_metric("SPQI", test_indices)

input_train = np.array([nsnr_train, sim_train, spqi_train]).transpose()
input_test = np.array([nsnr_test, sim_test, spqi_test]).transpose()

output_train, output_test = matlab.load_data_for_metric("InterNormRating", test_indices)

In [ ]:
regressor = model.make_vibromaf_pipeline()

regressor.fit(input_train, output_train)

In [ ]:
print_metric("Training Score R2", regressor.score(input_train, output_train))
print_metric("Test Score R2", regressor.score(input_test, output_test))

In [ ]:
output_prediction = regressor.predict(input_test)
print_mse_and_pc("vibromaf test", output_test / 10, output_prediction / 10)

In [ ]:
print_mse_and_pc("SPQI test", output_test / 10, spqi_test)
print_mse_and_pc("SIM test", output_test / 10, sim_test)
print_mse_and_pc("NSNR test", output_test / 10, nsnr_test)

In [ ]:
test_per_codec = matlab.split_per_codec(output_test)
vibromaf_per_codec = matlab.split_per_codec(output_prediction)
nsnr_per_codec = matlab.split_per_codec(nsnr_test)
sim_per_codec = matlab.split_per_codec(sim_test)
spqi_per_codec = matlab.split_per_codec(spqi_test)

print_mse_and_pc(
    "vibromaf test VC-PWQ", test_per_codec[0] / 10, vibromaf_per_codec[0] / 10
)
print_mse_and_pc("SPQI test VC-PWQ", test_per_codec[0] / 10, spqi_per_codec[0])
print_mse_and_pc("SIM test VC-PWQ", test_per_codec[0] / 10, sim_per_codec[0])
print_mse_and_pc("NSNR test VC-PWQ", test_per_codec[0] / 10, nsnr_per_codec[0])
print()

print_mse_and_pc(
    "vibromaf test PVC-SLP", test_per_codec[1] / 10, vibromaf_per_codec[1] / 10
)
print_mse_and_pc("SPQI test PVC-SLP", test_per_codec[1] / 10, spqi_per_codec[1])
print_mse_and_pc("SIM test PVC-SLP", test_per_codec[1] / 10, sim_per_codec[1])
print_mse_and_pc("NSNR test PVC-SLP", test_per_codec[1] / 10, nsnr_per_codec[1])
print()

print_mse_and_pc(
    "vibromaf test VPC-DS", test_per_codec[2] / 10, vibromaf_per_codec[2] / 10
)
print_mse_and_pc("SPQI test VPC-DS", test_per_codec[2] / 10, spqi_per_codec[2])
print_mse_and_pc("SIM test VPC-DS", test_per_codec[2] / 10, sim_per_codec[2])
print_mse_and_pc("NSNR test VPC-DS", test_per_codec[2] / 10, nsnr_per_codec[2])

In [ ]:
model.save_model(regressor, config.MODEL_PATH / "vibromaf-v1.pickle")

In [ ]:
def test_for_indices(test_indices):
    nsnr_train, nsnr_test = matlab.load_data_for_metric("NSNR", test_indices)
    sim_train, sim_test = matlab.load_data_for_metric("SIM", test_indices)
    spqi_train, spqi_test = matlab.load_data_for_metric("SPQI", test_indices)

    input_train = np.array([nsnr_train, sim_train, spqi_train]).transpose()
    input_test = np.array([nsnr_test, sim_test, spqi_test]).transpose()

    output_train, output_test = matlab.load_data_for_metric(
        "InterNormRating", test_indices
    )

    regressor = model.make_vibromaf_pipeline()

    regressor.fit(input_train, output_train)

    output_prediction = regressor.predict(input_test)
    print_mse_and_pc(
        f"vibromaf test {test_indices}", output_test / 10, output_prediction / 10
    )
    print()

In [ ]:
test_indices = list(itertools.combinations(range(8), 2))

for index in test_indices:
    test_for_indices(index)